In [1]:
from sklearn.feature_selection import SelectKBest, f_classif
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tslearn.utils import to_time_series_dataset
import biosppy.signals.ecg as ecg
import matplotlib.pyplot as plt
import neurokit2 as nk
import heartpy as hp
from statistics import mean
import functions
from multiprocessing import Pool
from datetime import time
import functions
import scipy
import math
import operator
from functions import *
import itertools
import scipy.stats as stats
import pyhrv.tools as tools
from pyhrv.hrv import hrv


In [2]:
folder = ""
raw = folder + "raw_data/"
# raw_dataset_x = pd.read_csv(raw + 'X_train.csv',index_col=['id'])


In [3]:
# raw_dataset_x.info()

In [4]:
# raw_dataset_y = pd.read_csv(raw + 'y_train.csv',index_col=['id'])
# raw_dataset_y.head()

In [5]:
#x_train, x_test, y_train, y_test = train_test_split(raw_dataset_x, raw_dataset_y, test_size=0.2, random_state=100)
x_train = pd.read_csv(raw + 'X_train.csv', index_col=['id'])
x_test = pd.read_csv(raw + 'X_test.csv', index_col=['id'])
y_train = pd.read_csv(raw + 'y_train.csv', index_col=['id'])
y_test = pd.DataFrame({'A': []})

In [ ]:
count_0 = 0
def apply_func(index,row):
    #print("Extracting features from row -> ",index)
    values_dict = {}
    a = datetime.datetime.now()
    ecg_signal = row.dropna().to_numpy()
    ecg_cleaned = nk.ecg_clean(ecg_signal, sampling_rate=300)
    #     ecg_cleaned = ecg_signal

    ecg_cleaned /= np.max(np.abs(ecg_cleaned), axis = 0)
    # to try
    #max_abs = max(ecg_cleaned.min(), ecg_cleaned.max(), key=abs)
    #ecg_cleaned = ecg_cleaned / max_abs
    #     # Extract R-peaks locations
    peaks, rpeaks = nk.ecg_peaks(ecg_cleaned, sampling_rate=300)
    # Delineate the ECG signal
    b = datetime.datetime.now()
    #print("r_peak", (b - a).seconds)
    
    nk_features = np.zeros(380)
    
    
    rpeaks_results = hrv(rpeaks['ECG_R_Peaks'], plot_ecg=False, show=False, plot_tachogram = False)
    values_dict['hrv_hr_mean'] = rpeaks_results['hr_mean']
    values_dict['hrv_hr_std'] = rpeaks_results['hr_std']
    values_dict['hrv_hr_max'] = rpeaks_results['hr_max']
    values_dict['hrv_hr_min'] = rpeaks_results['hr_min']
    values_dict['hrv_sdnn'] = rpeaks_results['sdnn']
    values_dict['hrv_sdnn_index'] = rpeaks_results['sdnn_index']
    values_dict['hrv_sdann'] = rpeaks_results['sdann']
    values_dict['hrv_rmssd'] = rpeaks_results['rmssd']
    values_dict['hrv_sdsd'] = rpeaks_results['sdsd']
    values_dict['hrv_pnn50'] = rpeaks_results['pnn50']
    values_dict['hrv_pnn20'] = rpeaks_results['pnn20']
    values_dict['hrv_nn50'] = rpeaks_results['nn50']
    values_dict['hrv_nn20'] = rpeaks_results['nn20']
    values_dict['hrv_sd1'] = rpeaks_results['sd1']
    values_dict['hrv_sd2'] = rpeaks_results['sd2']
    values_dict['hrv_sd_ratio'] = rpeaks_results['sd_ratio']
    #values_dict['hrv_dfa_short'] = rpeaks_results['dfa_short']
    #values_dict['hrv_dfa_long'] = rpeaks_results['dfa_long']
    values_dict['hrv_ellipse_area'] = rpeaks_results['ellipse_area']
    #values_dict['hrv_sample_entropy'] = rpeaks_results['sample_entropy']
    #values_dict['hrv_X_ratio'] = rpeaks_results['X_ratio']
    #values_dict['hrv_X_total'] = rpeaks_results['X_total']
    values_dict['hrv_fft_resampling_frequency'] = rpeaks_results['fft_resampling_frequency']
    values_dict['hrv_tinn_n'] = rpeaks_results['tinn_n']
    values_dict['hrv_tinn_m'] = rpeaks_results['tinn_m']
    values_dict['hrv_tinn'] = rpeaks_results['tinn']
    values_dict['hrv_tri_index'] = rpeaks_results['tri_index']
    
    #count values below 0 in ecg signal
    values_dict['below'] = np.count_nonzero(ecg_cleaned < 0)
    #count values above 0 in ecg signal
    values_dict['above'] = np.count_nonzero(ecg_cleaned > 0)



    if len(rpeaks['ECG_R_Peaks']) > 3:
        try:
            signal_dwt, waves_dwt = nk.ecg_delineate(ecg_cleaned, rpeaks, show_type='all', sampling_rate=300,
                                                     method="dwt")
            for key, arr in waves_dwt.items():
                if 'Onsets' in key or 'Offsets' in key:
                    array = np.array(arr)
                    #row = add_basic_info(row, np.array(arr), key)
                    #row = add_basic_info(row, amplitudes, f"{key}_amplitudes")
            nk_features = get_nk_features(ecg_cleaned, rpeaks['ECG_R_Peaks'], waves_dwt)
        except Exception as e:
            print("Signal flipped: ", e)
            ecg_signal = row.dropna().to_numpy()
            ecg_signal = -ecg_signal
            ecg_cleaned = nk.ecg_clean(ecg_signal, sampling_rate=300)
            peaks, rpeaks = nk.ecg_peaks(ecg_cleaned, sampling_rate=300)
            signal_dwt, waves_dwt = nk.ecg_delineate(ecg_cleaned, rpeaks, show_type='all', sampling_rate=300,
                                                     method="dwt")
            for key, arr in waves_dwt.items():
                if 'Onsets' in key or 'Offsets' in key:
                    array = np.array(arr)
                    #row = add_basic_info(row, np.array(arr), key)
            nk_features = get_nk_features(ecg_cleaned, rpeaks['ECG_R_Peaks'], waves_dwt)
    
    heart_rate = ecg.ecg(signal=ecg_cleaned, sampling_rate=300,show=False)['heart_rate']
    
    #heart rate features
    if(len(heart_rate) > 0):
        values_dict['heart_rate_mean'] = np.mean(heart_rate)
        values_dict['heart_rate_std'] = np.std(heart_rate)
        values_dict['heart_rate_max'] = np.max(heart_rate)
        values_dict['heart_rate_min'] = np.min(heart_rate)
        values_dict['heart_rate_median'] = np.median(heart_rate)
        values_dict['heart_rate_iqr'] = np.subtract(*np.percentile(heart_rate, [75, 25]))
        values_dict['heart_rate_skew'] = stats.skew(heart_rate)
        values_dict['heart_rate_kurtosis'] = stats.kurtosis(heart_rate)
        values_dict['heart_rate_var'] = np.var(heart_rate)
        values_dict['heart_rate_range'] = np.ptp(heart_rate)
    else:
        values_dict['heart_rate_mean'] = np.nan
        values_dict['heart_rate_std'] = np.nan
        values_dict['heart_rate_max'] = np.nan
        values_dict['heart_rate_min'] = np.nan
        values_dict['heart_rate_median'] = np.nan
        values_dict['heart_rate_iqr'] = np.nan
        values_dict['heart_rate_skew'] = np.nan
        values_dict['heart_rate_kurtosis'] = np.nan
        values_dict['heart_rate_var'] = np.nan
        values_dict['heart_rate_range'] = np.nan
    

    global count_0 
    count_0 += 378 - np.count_nonzero(nk_features)
    mu = np.zeros(180)
    median = np.zeros(180)
    std = np.zeros(180)
    heart_rate = np.zeros(180)
    if len(rpeaks['ECG_R_Peaks']) >= 2:
        # print(ecg.extract_heartbeats(signal, r_peaks, 300))
        nni = tools.nn_intervals(rpeaks['ECG_R_Peaks'])
        values_dict['nni_mean'] = np.mean(nni)
        values_dict['nni_std'] = np.std(nni)
        values_dict['nni_max'] = np.max(nni)
        values_dict['nni_min'] = np.min(nni)
        values_dict['nni_median'] = np.median(nni)
        values_dict['nni_iqr'] = np.subtract(*np.percentile(nni, [75, 25]))
        values_dict['nni_skew'] = stats.skew(nni)
        values_dict['nni_kurtosis'] = stats.kurtosis(nni)
        values_dict['nni_var'] = np.var(nni)
        values_dict['nni_range'] = np.ptp(nni)

        beats = ecg.extract_heartbeats(ecg_signal, rpeaks['ECG_R_Peaks'], 300)['templates']
        
        if len(beats) != 0:
            mu = np.mean(beats, axis=0)
            median = np.median(beats, axis=0)
            std = np.std(beats, axis=0)
    else:
        values_dict['nni_mean'] = np.nan
        values_dict['nni_std'] = np.nan
        values_dict['nni_max'] = np.nan
        values_dict['nni_min'] = np.nan
        values_dict['nni_median'] = np.nan
        values_dict['nni_iqr'] = np.nan
        values_dict['nni_skew'] = np.nan
        values_dict['nni_kurtosis'] = np.nan
        values_dict['nni_var'] = np.nan
        values_dict['nni_range'] = np.nan

    c = datetime.datetime.now()
    #print("extract_heartbeats", (c - b).seconds)

    d = datetime.datetime.now()
    #print("delineate", (d - c).seconds)
    fft_window = np.fft.fft(ecg_cleaned)
    fft_window_amplitude = np.abs(fft_window)
    fft_window_phase = np.angle(fft_window)
    fft_window_power = np.square(fft_window_amplitude)
    fft_freq = np.fft.fftfreq(len(ecg_cleaned), 1 / 300)
    index = 0
    for array in [fft_window_amplitude, fft_window_phase, fft_window_power, fft_freq]:
        values_dict[f"fft_mean_{index}"] = np.mean(array)
        values_dict[f"fft_median_{index}"] = np.median(array)
        values_dict[f"fft_std_{index}"] = np.std(array)
        values_dict[f"fft_min_{index}"] = np.min(array)
        values_dict[f"fft_max_{index}"] = np.max(array)
        values_dict[f"fft_skew_{index}"] = stats.skew(array)
        values_dict[f"fft_kurtosis_{index}"] = stats.kurtosis(array)
        values_dict[f"fft_iqr_{index}"] = np.subtract(*np.percentile(array, [75, 25]))
        values_dict[f"fft_q1_{index}"] = np.percentile(array, 25)
        values_dict[f"fft_q3_{index}"] = np.percentile(array, 75)
        values_dict[f"fft_q5_{index}"] = np.percentile(array, 5)
        values_dict[f"fft_q95_{index}"] = np.percentile(array, 95)
        values_dict[f"fft_q10_{index}"] = np.percentile(array, 10)
        values_dict[f"fft_q90_{index}"] = np.percentile(array, 90)
        values_dict[f"fft_mean_change_{index}"] = np.mean(np.diff(array))
        index+=1
    fft_window = np.fft.fft(median)
    fft_window_amplitude = np.abs(fft_window)
    fft_window_phase = np.angle(fft_window)
    fft_window_power = np.square(fft_window_amplitude)
    for array in [fft_window_amplitude, fft_window_phase, fft_window_power]:
        values_dict[f"fft_mean_{index}"] = np.mean(array)
        values_dict[f"fft_median_{index}"] = np.median(array)
        values_dict[f"fft_std_{index}"] = np.std(array)
        values_dict[f"fft_min_{index}"] = np.min(array)
        values_dict[f"fft_max_{index}"] = np.max(array)
        values_dict[f"fft_skew_{index}"] = stats.skew(array)
        values_dict[f"fft_kurtosis_{index}"] = stats.kurtosis(array)
        values_dict[f"fft_iqr_{index}"] = np.subtract(*np.percentile(array, [75, 25]))
        values_dict[f"fft_q1_{index}"] = np.percentile(array, 25)
        values_dict[f"fft_q3_{index}"] = np.percentile(array, 75)
        values_dict[f"fft_q5_{index}"] = np.percentile(array, 5)
        values_dict[f"fft_q95_{index}"] = np.percentile(array, 95)
        values_dict[f"fft_q10_{index}"] = np.percentile(array, 10)
        values_dict[f"fft_q90_{index}"] = np.percentile(array, 90)
        values_dict[f"fft_mean_change_{index}"] = np.mean(np.diff(array))
        index+=1
    # freq = np.fft.fftfreq(np.array(ecg_cleaned).shape[-1], d=1/300)
    # fft_ps = np.abs(fft_window)**2
    
    #print(fft_window, freq, fft_ps)
    e = datetime.datetime.now()
    #print("fft", (e - d).seconds)
    row = row.drop(columns=[f'x{num}' for num in range(17807)])
    f = datetime.datetime.now()
    
    #print("drop", (f - e).seconds)
    for i in range(180):
        values_dict[f'mu_{i}'] = mu[i]
        values_dict[f'median_{i}'] = median[i]
        values_dict[f'std_{i}'] = std[i]
    # for i in range(len(fft_window)):
    #     #values_dict[f'fft_window_{i}'] = fft_window[i]
    #     #values_dict[f'freq_{i}'] = freq[i]
    #     #values_dict[f'fft_ps_{i}'] = fft_ps[i]
    #     pass
    for i in range(len(nk_features)):
        values_dict[f'nk_feat_{i}'] = nk_features[i]
    
    #row.append(pd.Series(values_dict))
    g = datetime.datetime.now()
    #print("add", (g - f).seconds)
    return values_dict

In [15]:
#classify ecg
from functions import *
def classify_ecg(x_train):
    x_train_ecg = x_train.copy()
    '''
    x_train_ecg = x_train_ecg.apply(apply_func)
    '''
    new_train_dict = {}
    for index,row in x_train.iterrows():
        new_train_dict[str(index)] = apply_func(index,row)
        if len(new_train_dict) % 500 == 0:
            print(len(new_train_dict))
        
    return pd.DataFrame.from_dict(new_train_dict,orient='index')

In [ ]:
x_train = classify_ecg(x_train)

C:\Users\david\anaconda3\envs\preprocessing.ipynb\lib\site-packages\pyhrv\time_domain.py:576: UserWarning: CAUTION: The TINN computation is currently providing incorrect results in the most cases due to a malfunction of the function. This function will be reviewed over the next updates to solve this issue
  warnings.warn('CAUTION: The TINN computation is currently providing incorrect results in the most cases due to a '
C:\Users\david\anaconda3\envs\preprocessing.ipynb\lib\site-packages\pyhrv\utils.py:367: UserWarning: Signal duration is to short for segmentation into 300000s. Input data will be returned.
  warnings.warn("Signal duration is to short for segmentation into %is. Input data will be returned." % duration)
C:\Users\david\anaconda3\envs\preprocessing.ipynb\lib\site-packages\pyhrv\time_domain.py:303: UserWarning: Signal duration too short for SDANN computation.
  warnings.warn("Signal duration too short for SDANN computation.")
C:\Users\david\anaconda3\envs\preprocessing.ipynb

C:\Users\david\anaconda3\envs\preprocessing.ipynb\lib\site-packages\pyhrv\utils.py:367: UserWarning: Signal duration is to short for segmentation into 300000s. Input data will be returned.
  warnings.warn("Signal duration is to short for segmentation into %is. Input data will be returned." % duration)
C:\Users\david\anaconda3\envs\preprocessing.ipynb\lib\site-packages\pyhrv\time_domain.py:303: UserWarning: Signal duration too short for SDANN computation.
  warnings.warn("Signal duration too short for SDANN computation.")
C:\Users\david\anaconda3\envs\preprocessing.ipynb\lib\site-packages\pyhrv\time_domain.py:576: UserWarning: CAUTION: The TINN computation is currently providing incorrect results in the most cases due to a malfunction of the function. This function will be reviewed over the next updates to solve this issue
  warnings.warn('CAUTION: The TINN computation is currently providing incorrect results in the most cases due to a '
C:\Users\david\anaconda3\envs\preprocessing.ipynb

C:\Users\david\anaconda3\envs\preprocessing.ipynb\lib\site-packages\pyhrv\time_domain.py:576: UserWarning: CAUTION: The TINN computation is currently providing incorrect results in the most cases due to a malfunction of the function. This function will be reviewed over the next updates to solve this issue
  warnings.warn('CAUTION: The TINN computation is currently providing incorrect results in the most cases due to a '
C:\Users\david\anaconda3\envs\preprocessing.ipynb\lib\site-packages\pyhrv\nonlinear.py:308: UserWarning: Not enough NNI samples for Detrended Fluctuations Analysis.
  warnings.warn("Not enough NNI samples for Detrended Fluctuations Analysis.")
C:\Users\david\anaconda3\envs\preprocessing.ipynb\lib\site-packages\pyhrv\utils.py:367: UserWarning: Signal duration is to short for segmentation into 300000s. Input data will be returned.
  warnings.warn("Signal duration is to short for segmentation into %is. Input data will be returned." % duration)
C:\Users\david\anaconda3\envs

C:\Users\david\anaconda3\envs\preprocessing.ipynb\lib\site-packages\pyhrv\nonlinear.py:308: UserWarning: Not enough NNI samples for Detrended Fluctuations Analysis.
  warnings.warn("Not enough NNI samples for Detrended Fluctuations Analysis.")
C:\Users\david\anaconda3\envs\preprocessing.ipynb\lib\site-packages\pyhrv\utils.py:367: UserWarning: Signal duration is to short for segmentation into 300000s. Input data will be returned.
  warnings.warn("Signal duration is to short for segmentation into %is. Input data will be returned." % duration)
C:\Users\david\anaconda3\envs\preprocessing.ipynb\lib\site-packages\pyhrv\time_domain.py:303: UserWarning: Signal duration too short for SDANN computation.
  warnings.warn("Signal duration too short for SDANN computation.")
C:\Users\david\anaconda3\envs\preprocessing.ipynb\lib\site-packages\pyhrv\time_domain.py:576: UserWarning: CAUTION: The TINN computation is currently providing incorrect results in the most cases due to a malfunction of the funct

C:\Users\david\anaconda3\envs\preprocessing.ipynb\lib\site-packages\pyhrv\nonlinear.py:308: UserWarning: Not enough NNI samples for Detrended Fluctuations Analysis.
  warnings.warn("Not enough NNI samples for Detrended Fluctuations Analysis.")
C:\Users\david\anaconda3\envs\preprocessing.ipynb\lib\site-packages\pyhrv\utils.py:367: UserWarning: Signal duration is to short for segmentation into 300000s. Input data will be returned.
  warnings.warn("Signal duration is to short for segmentation into %is. Input data will be returned." % duration)
C:\Users\david\anaconda3\envs\preprocessing.ipynb\lib\site-packages\pyhrv\time_domain.py:303: UserWarning: Signal duration too short for SDANN computation.
  warnings.warn("Signal duration too short for SDANN computation.")
C:\Users\david\anaconda3\envs\preprocessing.ipynb\lib\site-packages\pyhrv\time_domain.py:576: UserWarning: CAUTION: The TINN computation is currently providing incorrect results in the most cases due to a malfunction of the funct

...

C:\Users\david\anaconda3\envs\preprocessing.ipynb\lib\site-packages\pyhrv\time_domain.py:576: UserWarning: CAUTION: The TINN computation is currently providing incorrect results in the most cases due to a malfunction of the function. This function will be reviewed over the next updates to solve this issue
  warnings.warn('CAUTION: The TINN computation is currently providing incorrect results in the most cases due to a '
C:\Users\david\anaconda3\envs\preprocessing.ipynb\lib\site-packages\pyhrv\nonlinear.py:308: UserWarning: Not enough NNI samples for Detrended Fluctuations Analysis.
  warnings.warn("Not enough NNI samples for Detrended Fluctuations Analysis.")
C:\Users\david\anaconda3\envs\preprocessing.ipynb\lib\site-packages\pyhrv\utils.py:367: UserWarning: Signal duration is to short for segmentation into 300000s. Input data will be returned.
  warnings.warn("Signal duration is to short for segmentation into %is. Input data will be returned." % duration)
C:\Users\david\anaconda3\envs

C:\Users\david\anaconda3\envs\preprocessing.ipynb\lib\site-packages\pyhrv\time_domain.py:576: UserWarning: CAUTION: The TINN computation is currently providing incorrect results in the most cases due to a malfunction of the function. This function will be reviewed over the next updates to solve this issue
  warnings.warn('CAUTION: The TINN computation is currently providing incorrect results in the most cases due to a '
C:\Users\david\anaconda3\envs\preprocessing.ipynb\lib\site-packages\pyhrv\nonlinear.py:308: UserWarning: Not enough NNI samples for Detrended Fluctuations Analysis.
  warnings.warn("Not enough NNI samples for Detrended Fluctuations Analysis.")
C:\Users\david\anaconda3\envs\preprocessing.ipynb\lib\site-packages\pyhrv\utils.py:367: UserWarning: Signal duration is to short for segmentation into 300000s. Input data will be returned.
  warnings.warn("Signal duration is to short for segmentation into %is. Input data will be returned." % duration)
C:\Users\david\anaconda3\envs

C:\Users\david\anaconda3\envs\preprocessing.ipynb\lib\site-packages\pyhrv\time_domain.py:576: UserWarning: CAUTION: The TINN computation is currently providing incorrect results in the most cases due to a malfunction of the function. This function will be reviewed over the next updates to solve this issue
  warnings.warn('CAUTION: The TINN computation is currently providing incorrect results in the most cases due to a '
C:\Users\david\anaconda3\envs\preprocessing.ipynb\lib\site-packages\pyhrv\nonlinear.py:308: UserWarning: Not enough NNI samples for Detrended Fluctuations Analysis.
  warnings.warn("Not enough NNI samples for Detrended Fluctuations Analysis.")
C:\Users\david\anaconda3\envs\preprocessing.ipynb\lib\site-packages\pyhrv\utils.py:367: UserWarning: Signal duration is to short for segmentation into 300000s. Input data will be returned.
  warnings.warn("Signal duration is to short for segmentation into %is. Input data will be returned." % duration)
C:\Users\david\anaconda3\envs

C:\Users\david\anaconda3\envs\preprocessing.ipynb\lib\site-packages\pyhrv\time_domain.py:576: UserWarning: CAUTION: The TINN computation is currently providing incorrect results in the most cases due to a malfunction of the function. This function will be reviewed over the next updates to solve this issue
  warnings.warn('CAUTION: The TINN computation is currently providing incorrect results in the most cases due to a '
C:\Users\david\anaconda3\envs\preprocessing.ipynb\lib\site-packages\pyhrv\nonlinear.py:308: UserWarning: Not enough NNI samples for Detrended Fluctuations Analysis.
  warnings.warn("Not enough NNI samples for Detrended Fluctuations Analysis.")
C:\Users\david\anaconda3\envs\preprocessing.ipynb\lib\site-packages\pyhrv\time_domain.py:576: UserWarning: CAUTION: The TINN computation is currently providing incorrect results in the most cases due to a malfunction of the function. This function will be reviewed over the next updates to solve this issue
  warnings.warn('CAUTION:

C:\Users\david\anaconda3\envs\preprocessing.ipynb\lib\site-packages\pyhrv\time_domain.py:576: UserWarning: CAUTION: The TINN computation is currently providing incorrect results in the most cases due to a malfunction of the function. This function will be reviewed over the next updates to solve this issue
  warnings.warn('CAUTION: The TINN computation is currently providing incorrect results in the most cases due to a '
C:\Users\david\anaconda3\envs\preprocessing.ipynb\lib\site-packages\pyhrv\nonlinear.py:308: UserWarning: Not enough NNI samples for Detrended Fluctuations Analysis.
  warnings.warn("Not enough NNI samples for Detrended Fluctuations Analysis.")
C:\Users\david\anaconda3\envs\preprocessing.ipynb\lib\site-packages\pyhrv\time_domain.py:576: UserWarning: CAUTION: The TINN computation is currently providing incorrect results in the most cases due to a malfunction of the function. This function will be reviewed over the next updates to solve this issue
  warnings.warn('CAUTION:

In [ ]:
x_train.to_csv('x_train_preprocess.csv')
x_train.to_csv('x_train_preprocess_davide.csv')

In [ ]:
x_test = classify_ecg(x_test)

In [ ]:
x_test.to_csv('x_test_preprocess.csv')
x_test.to_csv('x_test_preprocess_davide.csv')

In [ ]:
#count nan values per column
def count_nan(df):
    for col in df.columns:
        if(df[col].isna().sum() > 0):
            print(col,df[col].isna().sum())
    
count_nan(x_train)

In [ ]:
#print array of columns names
def print_columns(df):
    for col in df.columns:
        print("\'",col,"\',",sep='',end='')
print_columns(x_train)